이 노트북은 개인 공부용 노트북이다.  
이 노트북에서는 Neural Network의 구축에 대해 저장할 것이다.  
사용할 데이터는 PTS Nov 2021이다.  


Last Update: 21.11.23

가장 궁금한 것은 NN의 하이퍼파라미터 튜닝이다. 러닝레이트, 레이어의 개수, 레이어의 크기, 드랍아웃과 배치노멀라이즈 등 조절해야할 파라미터의 수가 너무 많기 때문에
다른 머신러닝 모델에 비해 튜닝하는 난이도가 훨씬 높다고 생각한다. TPS Nov 2021에서 많은 Vote를 받은 커널들의 저자들의 답변은 아래와 같았다.

1. I had experience with the same from one of my previous competitions and i experimented a bit on the same neural net i changed hidden units based on the results i was receiving in the CV. Most of the part was trial and error
이전 대회에서도 같은 경험을 했고 실험도 좀 해봤어요.
같은 신경망에서 CV에서 받은 결과에 따라 숨겨진 단위를 바꿨어요
대부분은 시행착오였습니다.

2. Hi, I did this by manually tuning the hyperparams and constantly checking with my cv_validation score. Mostly I checked for overfitting (see the graphs in "Evaluation")  
안녕하세요. 저는 수동으로 하이퍼패람을 튜닝하고 지속적으로 cv_validation 점수를 확인하여 이 작업을 수행했습니다. 대부분 과적합 여부를 확인했습니다("평가"의 그래프 참조).

3. There is not trick here just trial and error  
여기에는 속임수가 없습니다. 시행착오만 있을 뿐.  
  
이들의 공통점은 대게 잘 알려진 하이퍼 파라미터들을 사용한 것과 직접 시행착오를 통해 최적화한 것이다. 또한 교차검증을 통해 훈련점수와 검증점수를 모두 기록하고 그래프를 통해 과적합 유무를 확인했다는 것과 과적합이 없는 모델 중 가장 높은 교차점수를 갖는 모델을 선택했다는 것이다. 주어진 훈련데이터를 모두 사용했으며 KFold는 모두 5를 사용했다.

다음은 은닉층의 크기를 직감적으로 산정할 수 있는 기준 중 하나이다(아직 확실하지 않다).  
refer1: https://kugancity.tistory.com/entry/MLP%EC%97%90%EC%84%9C-%EC%A0%81%EC%A0%88%ED%95%9C-hidden-unit-%EA%B0%9C%EC%88%98-%EC%82%B0%EC%A0%95%ED%95%98%EA%B8%B0  
refer2: https://m.blog.naver.com/laonple/220576650094  
  
Manual:  
이 방법은 경험을 기반으로 좋을 것 같은 파라미터들을 시도해보는 것이다. 아래는 refer1에 언급된 노하우들인데 확실하지 않으므로 참고만 한다.
1. The number of hidden neurons shold be between the size of the input layer and the size of the output layer.  
숨겨진 뉴런의 수는 입력층의 크기와 출력층의 크기 사이에 있습니다.
2. The number of hidden neurons should be 2/3 the size of the input layer, plus the size of the output layer.  
숨겨진 뉴런의 수는 입력 층의 2/3 크기에 출력 층의 크기를 더해야 합니다.
3. The number of hidden neurons should be less than twice the size of he input layer.  
숨겨진 뉴런의 수는 입력층의 두 배 미만이어야 합니다.  
  

GridSearch:  
Manual과 비슷한 방법인데, 주어진 조합을 모두 수행하는 것이다. Manual과 GridSearch는 validation data가 필요하다.  
  
RandomSearch:  
주어진 범위 내에서 랜덤한 조합을 시도하며, GridSearch보다 빠르게 옵티멀한 조합을 찾는 경향이 있다고 한다.  
  
Bayesian optimization:  
탐색 과정에서 얻은 결과를 활용하여 최적값을 찾아가는 방법으로 위 방법들에 비해 효율성을 강조한다.

## NN Baseline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
train = pd.read_csv('../input/tabular-playground-series-nov-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-nov-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

In [4]:
# Data load and Feature Engineering

X_train = train.drop(['id', 'target'], axis=1).copy()
y_train = train.target
X_test = test.drop(['id'], axis=1).copy()

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [9]:
# callback functions

es = EarlyStopping(
    monitor='val_loss',
    patience=20,
    verbose=0,
    mode='min',
    restore_best_weights=True
)

plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=.2,
    patience=5,
    verbose=0,
    mode='min'
)

In [10]:
# generate NN model

def get_model():
    input_layer = Input(shape=X_test.shape[1], name='input')
    h1 = Dense(128, activation='swish', name='Dense1')(input_layer)
    h2 = Dense(64, activation='swish', name='Dense2')(h1)
    h3 = Dense(32, activation='swish', name='Dense3')(h2)
    output_layer = Dense(1, activation='sigmoid', name='output')(h3)
    model = Model(inputs=input_layer, outputs=output_layer, name='baseline')
    
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='binary_crossentropy',
        metrics=['AUC'])
    
    return model

In [11]:
SEED = 42
BATCH_SIZE = 2048
EPOCHS = 2000
VERBOSE = 0

In [12]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
histories = {}
valid_mean = []
y_preds = []
for i, (train_idx, valid_idx) in enumerate(cv.split(X_train, y_train)):
    xtrain, xvalid = X_train.iloc[train_idx, :], X_train.iloc[valid_idx, :]
    ytrain, yvalid = y_train[train_idx], y_train[valid_idx]
    
    base_model = get_model()
    history = base_model.fit(xtrain, ytrain,
                            validation_data = (xvalid, yvalid),
                            callbacks=(es, plateau), verbose=VERBOSE,
                            epochs=EPOCHS, batch_size=BATCH_SIZE)
    histories[i] = history
    
    y_valid_pred  = base_model.predict(xvalid).reshape(1, -1)[0]
    valid_score = roc_auc_score(yvalid, y_valid_pred)
    valid_mean.append(valid_score)
    print('*'*40)
    print(f'Fold {i + 1} | Traning and Evaluate Validation')
    print(f'AUC of validation: {valid_score}')
    print('*'*40+'\n')
    
    y_test_pred = base_model.predict(X_test).reshape(1, -1)[0]
    y_preds.append(y_test_pred)
print('Fold Finished')
print(f'AUC MEAN: {np.array(valid_mean).mean()}')
print(f'AUC STD: {np.array(valid_mean).std()}')

In [44]:
for i in range(5):
    f, ax = plt.subplots(1, 2, figsize=(20, 3))
    sns.lineplot(data=histories[i].history['loss'], ax=ax[0], label='train loss')
    sns.lineplot(data=histories[i].history['val_loss'], ax=ax[0], label='validation loss')
    sns.lineplot(data=histories[i].history['auc'], ax=ax[1], label='train auc')
    sns.lineplot(data=histories[i].history['val_auc'], ax=ax[1], label='validation auc')
    f.suptitle(f'FOLD {i + 1}')

In [55]:
# submission
y_pred = np.mean(np.column_stack(y_preds), axis=1)
sub.target = y_pred
sub.to_csv('nn baseline submission.csv', index=False)